In [1]:
import logging
import sys

# Log file location
logfile = 'debug.log'
# Define the log format
log_format = (
    '[%(asctime)s] %(levelname)-8s %(name)-12s %(message)s')

# Define basic configuration
logging.basicConfig(
    # Define logging level
    level=logging.DEBUG,
    # Declare the object we created to format the log messages
    format=log_format,
    # Declare handlers
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout),
    ]
)

# Define your own logger name
log = logging.getLogger(__name__)

In [2]:
log.debug('This is a debug message')
log.info('This is an info message')
log.warning('This is a warning message')
log.error('This is an error message')
log.critical('This is a critical message')

[2021-06-10 11:49:53,267] DEBUG    __main__     This is a debug message
[2021-06-10 11:49:53,273] INFO     __main__     This is an info message
[2021-06-10 11:49:53,276] WARNING  __main__     This is a warning message
[2021-06-10 11:49:53,278] ERROR    __main__     This is an error message
[2021-06-10 11:49:53,279] CRITICAL __main__     This is a critical message
